In [0]:
!pip install tqdm --upgrade

!pip install geopy --upgrade

In [0]:
import pandas as pd
import numpy as np
np.random.seed(0)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'HMRNN/'

Mounted at /content/gdrive


In [0]:
df = pd.read_csv(base_dir + "smalldata.csv", sep='\t')
venue_city = dict()

In [0]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="specify_your_app_name_here", timeout=None)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=.5)

In [0]:
location = geocode("40.745837, -73.987993")

In [0]:
print(location)

In [0]:
geolocator = Nominatim(user_agent="POI",timeout=None)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=.5)

import pickle

isTimeOut = False

def getCity(l):
    global venue_city, isTimeOut
    if(isTimeOut) return np.nan
    if(l['venue_id'] in venue_city.keys()):
        #print(venue_city[l['venue_id']])
        return venue_city[l['venue_id']]
    try:
        venue_city[l['venue_id']] = geocode(f"{l['latitude']}, {l['longitude']}").raw['address']['city']
        f = open(base_dir + "citylist.pkl", "wb")
        pickle.dump(venue_city, f)
        f.close()
        print(venue_city[l['venue_id']])
        return venue_city[l['venue_id']]
    except Exception:
        return np.nan

In [0]:
######################## Only for threading
import threading
import time

def run(df):
    df['city_id'] = df.progress_apply(getCity, axis=1)

In [0]:
from multiprocessing import  Pool
import io
from tqdm import tqdm
tqdm.pandas()

#df['city_id'] = df.progress_apply(getCity, axis=1)
def parallelize_dataframe(df, func, n_cores=6):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

df = parallelize_dataframe(df, run)

In [0]:
len(venue_city)

0

In [14]:
f = open(base_dir + "citylist.pkl", "rb")
d = pickle.load(f)
f.close()
d

FileNotFoundError: ignored

In [4]:
df.head()

,user_id,venue_id,utc_time,latitude,longitude,category,country_code,country_code_id
0,260,31114,22,40.745837,-73.987993,198,US,60
1,260,51008,28,40.760108,-74.044416,253,US,60
2,260,28061,58,40.867217,-73.896747,76,US,60
3,260,35026,90,40.745878,-73.988142,255,US,60
4,260,48877,98,40.757713,-74.043751,410,US,60


In [0]:
df.to_csv(base_dir+"smalldata.csv", sep='\t', index=False)